In [28]:
import os
import json
import numpy as np
from openai import OpenAI
import os
import json
import numpy as np
import openai
import traceback
from dotenv import load_dotenv
import os
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch
import pandas as pd

# .env 파일 로드
load_dotenv('.env')
# API_KEY 값을 가져옴
openai_key = os.getenv('OPENAI_TEAM_API_KEY')
client_gpt = openai.OpenAI(api_key=openai_key)

In [29]:
datas = pd.read_csv('/root/KRX_LLM/data/stock_dataset_5000.csv', encoding='utf-8')


In [30]:
def generate_explanation_from_stock(data):
    """
    Generates explanation for stock data using GPT-4o-mini model.
    """
    # Extract context fields

    # You are a skilled question generator focusing on news content. Using the provided title, dates, and content, generate exactly 5 high-quality question-answer pairs in Korean. Each question should focus on expanding the reader's understanding of the news context. Each question should align with the following categories to capture important aspects of news content: summary, main points, specific details, real-world implications, and broader significance.

    # Define prompt to ensure clear QA pair generation around the keyword and context
    
    stock = data['input_text']
    
    
    prompt = f"""
    You are a skilled stock market analyst. Using the provided stock data, analyze and determine whether the expected trend is an increase ([증가]) or a decrease ([감소]). Generate a detailed explanation in Korean based on your analysis, and specify whether the trend indicates an expected increase or decrease. Your explanation should cite specific data points to support your conclusion.

    If any data points suggest a deviation from the inferred trend, acknowledge these points and provide reasons why they might represent short-term fluctuations, or offer context as to why they may not impact the overall trend.

    The stock data is as follows:
    {stock}

    ### Explanation:
    - Start by analyzing key indicators such as closing prices, adjusted closing prices, and incremental values (e.g., `inc-5`, `inc-10`).
    - Determine and state the inferred trend ([증가] or [감소]) based on the data.
    - Provide specific data points that support this trend.
    - Acknowledge any short-term deviations and justify why they may not affect the inferred trend.

    Please respond in Korean, providing a comprehensive and balanced explanation of the expected trend and supporting rationale.
    
    ### JSON Output:
    Respond in the following JSON format:
    json
    {{
    "expectation": "<[증가] or [감소], based on your analysis>",
    "explanation": "<Detailed explanation in Korean, based on the stock data, supporting your chosen expectation>"
    }}
    """
    
    response = client_gpt.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{'role': 'user', 'content': prompt}],
        response_format = { 'type': "json_object" }
    )
    
    # Process the response
    qa_output = response.choices[0].message.content
    
    return qa_output


In [31]:
datas = datas[3750: 4000]

In [32]:
results = []
for i in range(len(datas)):
    data = datas.iloc[i]
    try:
        result = generate_explanation_from_stock(data)
        result = json.loads(result)
    except Exception as e:
        continue

    result = generate_explanation_from_stock(data)
    result = json.loads(result)
    
    if result['expectation'] == data['answer']:
        result['stock'] = data['input_text']
        results.append(result)


    print('설명: ' + result['explanation'])
    print('실제값: ' + data['answer'])
    print('예측값: ' + result['expectation'])

설명: 주식 STOCK_3751의 분석 결과를 바탕으로 보면, 전체적인 주가 흐름은 하락세를 보이고 있습니다. 특히 최근 30일의 거래 데이터를 살펴보면, 'close' 가격이 2020-07-30일 기준으로 -0.1로 나타나며, 그 이전의 가격보다 낮은 상태입니다. 또한, 'adj-close'도 -0.1으로 동일한 양상을 보이고 있습니다.

'와 관련하여 'inc-5', 'inc-10', 'inc-15', 'inc-20', 'inc-25', 'inc-30' 값을 보면, 대부분의 수치가 음수로 나타나 주식이 하락하고 있음을 시사합니다. 예를 들어, 'inc-5'는 -1.5, 'inc-10'은 -9.2로 하락폭이 상당히 큽니다. 

특히, 2020-07-28일 알림처럼, 주가는 -6.4로 가장 낮고 이러한 하락세가 지속되고 있습니다. 이러한 흐름은 단기적인 변동이 아닌 지속적인 하락을 나타내며, 향후 주가가 회복될 가능성은 낮다고 판단됩니다.

물론 2020-07-17일에는 3.3으로 가장 높은 주가를 기록하였고, 이후 며칠간의 반등이 있었지만, 이는 일시적인 회복으로 보이며, 지속적으로 하락세를 보이고 있는 현재 상황에서는 큰 영향을 미치지 않을 것입니다. 따라서 STOCK_3751의 주가에 대한 예측은 [감소]로 설정하는 것이 적절합니다.
실제값: [감소]
예측값: [감소]
설명: STOCK_3752의 주식 데이터 분석 결과, 향후 주가는 상승할 것으로 예상됩니다. 주가에서 가장 중요한 지표인 종가(closing price)와 조정 종가(adjusted close price)를 살펴보면, 2022년 4월 26일에 주가는 7.1로 마감하였고, 조정 종가는 동일하게 7.1로 나타나고 있습니다. 초기 데이터인 2022년 4월 13일에는 종가가 -1.1 이었던 점과 비교했을 때 매우 긍정적인 경향을 보입니다.

또한, 최근 5일간의 증가는 9.2로 매우 높고, 10일, 15일, 20일 간의 변화에 대해서도 각각 1.3, 1.7, 4.6, 5.5로 상당한 상승세를 나타

In [34]:
df_results = pd.DataFrame(results)
df_results.to_csv('/root/KRX_LLM/outputs/stock_result1.csv', index=False)